In [75]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling Main.sc


path: String = "/home/kunmok/Git/chisel-bootcamp/source/load-ivy.sc"

In [63]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

In [74]:
import breeze.numerics.{atan, pow, sqrt}
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in1     = Input(UInt(3.W))
    val in2     = Input(UInt(3.W))
    val out     = Output(Vec(6, UInt(3.W)))
    val out1    = Output(UInt(log2Ceil(6).W))
//     val out1    = Output(Vec(3, UInt(log2Ceil(6).W)))
//     val out2    = Output(Vec(3, Vec(2, UInt(1.W))))
  })
   
  def dec2bitarray(genPolynoimial: List[Int], bit_width: Int) = {
    val numRow = genPolynoimial.length
    val bitarray = Array.ofDim[Int](numRow, bit_width)
    for (i <- 0 until numRow){
      val binary_string = genPolynoimial(i).toBinaryString
      val len = binary_string.length
      for (j <- 0 until bit_width){
        bitarray(i)(bit_width-j-1) = binary_string(len-j-1).asDigit
      }
    }
    bitarray
  }

  def findMinBitWidth(arg: List[Int]): Int = {
    val result: Int = math.ceil(math.log10(arg.max)/math.log10(2)).asInstanceOf[Int]
    result
  }

  def onesInPuncMat(arg: Array[Array[Int]]): Unit ={
    arg.map(breeze.linalg.Vector(_)).reduce(_ + _)
  }  
//     val shiftReg = RegInit(Vec(Seq.fill(params.K)(0.U(1.W))))
    val n = 2
    val O = 6 
    val K = 3 
    val m = K - 1
    
    val punctureMatrix = List(6,5)
    val puncMatBitWidth   = findMinBitWidth(punctureMatrix)
    println(puncMatBitWidth)
    val punctureList      = dec2bitarray(punctureMatrix, puncMatBitWidth)
    println(punctureList(0).toList)
    println(punctureList(1).toList)
    val punctureVec       = Wire(Vec(n, Vec(puncMatBitWidth, UInt(1.W))))
    (0 until n).map(i => {
      (0 until puncMatBitWidth).map(j => {
        punctureVec(i)(j) := (punctureList(i)(j)).U
      })
    })
    val puncListColSum   = punctureList.map(breeze.linalg.Vector(_)).reduce(_ + _)
    println(puncListColSum)
    val puncPolyWire = Wire(Vec(puncMatBitWidth, UInt((log2Ceil(n+1)).W)))
    (0 until puncMatBitWidth).map(i => { puncPolyWire(i) := puncListColSum(i).U })
    val puncIndices = punctureList.scanLeft(Array.fill(punctureList(0).length)(0)) ((x,y) =>
    x.zip(y).map(e => e._1 + e._2)).drop(1)
    println(puncIndices(0).toList)
    println(puncIndices(1).toList)
  // convert this to chisel-usable variable using vector of wires
    val puncIndicesWire = Wire(Vec(n, Vec(puncMatBitWidth, UInt(1.W))))
    (0 until n).map(i => {
      (0 until puncMatBitWidth).map(j => {
        puncIndicesWire(i)(j) := puncIndices(i)(j).U
      })
    })
    val o_cnt             = RegInit(0.U(log2Ceil(O).W))  // counter for data vector tracker
    val p_cnt             = RegInit(0.U(log2Ceil(O).W))  // counter for outReg tracker
    val sStartRecv  = 0.U(1.W)        // start taking input bits
    val sDone        = 1.U(1.W)
    val state = RegInit(0.U(1.W))
    val bufInterleaver             = RegInit(VecInit(Seq.fill(O)(0.U(3.W))))  // buffer for interleaver
    val AXWires           = Wire(Vec(n, Vec(K, UInt(3.W))))
    AXWires(0)(0) := 1.U
    AXWires(0)(1) := 1.U
    AXWires(0)(2) := io.in1
    AXWires(1)(0) := 1.U
    AXWires(1)(1) := 1.U
    AXWires(1)(2) := io.in2
    
    // start puncturing 
    for (i <- 0 until n) {
      when(punctureVec((o_cnt+i.U) % n.U)((o_cnt / n.U) % (puncMatBitWidth.U)) === 1.U) {
bufInterleaver(p_cnt - 1.U + puncIndicesWire((o_cnt+i.U) % n.U)(((o_cnt+i.U) / n.U) % (puncMatBitWidth.U))) := AXWires((o_cnt + i.U) % n.U)(m.U)
      }
    }
    p_cnt := p_cnt + puncPolyWire((o_cnt/(n.U)) % puncMatBitWidth.U)
    o_cnt := o_cnt + (n).U

    io.out := bufInterleaver
//     io.out1 := puncPolyWire
    io.out1 := p_cnt 
    
}

class MyOperatorsTester(c: MyOperators) extends PeekPokeTester(c) {
  poke(c.io.in1, 3)  //cnt == 0.U
  poke(c.io.in2, 1)
//   // expect 0,0,0,0,0,0
//   // right 
  expect(c.io.out(0), 0)
  expect(c.io.out(1), 0)
  expect(c.io.out(2), 0)
  expect(c.io.out(3), 0)
  expect(c.io.out(4), 0)
  expect(c.io.out(5), 0)
  expect(c.io.out1, 0)
    
//   // expect 1,1,0,0,0,0
  step(1)  //cnt == 1
//   expect(c.io.out(0), 3)
//   expect(c.io.out(1), 1) 
//   expect(c.io.out(2), 0)
//   expect(c.io.out(3), 0)
//   expect(c.io.out(4), 0)
//   expect(c.io.out(5), 0)
  expect(c.io.out1, 2)
    
//   // expect 1,1,1,0,0,0
  step(1)  // wrong 
//   expect(c.io.out(0), 3)
//   expect(c.io.out(1), 1)
//   expect(c.io.out(2), 3)
//   expect(c.io.out(3), 0)
//   expect(c.io.out(4), 0)
//   expect(c.io.out(5), 0)
  expect(c.io.out1, 3)
    step(1)
    expect(c.io.out1, 4)
    step(1)
    expect(c.io.out1, 6)
    step(4)
    expect(c.io.out1, 4)
//     step(1)
//     expect(c.io.out1, 8)
// //   // expect 1,1,1,0,0,1
//   step(1)
//   expect(c.io.out(0), 3)
//   expect(c.io.out(1), 1)
//   expect(c.io.out(2), 3)
//   expect(c.io.out(3), 1)
//   expect(c.io.out(4), 0)
//   expect(c.io.out(5), 0)
    
}
assert(Driver(() => new MyOperators) {c => new MyOperatorsTester(c)})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
3
List(1, 1, 0)
List(1, 0, 1)
DenseVector(2, 1, 1)
List(1, 1, 0)
List(2, 1, 1)
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 35.9 ms
Total FIRRTL Compile Time: 36.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1541007290462
test cmd73WrapperHelperMyOperators Success: 12 tests passed in 13 cycles taking 0.015893 seconds
[info] [0.013] RAN 8 CYCLES PASSED
SUCCESS!!


import breeze.numerics.{atan, pow, sqrt}

defined class MyOperators
defined class MyOperatorsTester